---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
# import matplotlib.pyplot as plt

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def distribution(G):
    deg = G.degree()
    vals = sorted(set(deg.values()))
    hist = [list(deg.values()).count(j)/float(nx.number_of_nodes(G)) for j in vals]
    
#     plt.plot(vals, hist, 'o')
#     plt.xscale('log')
#     plt.yscale('log')
#     plt.show()
    return hist

def graph_identification():
    answer = []
    
    for G in P1_Graphs:
        distrib_hist = distribution(G)
        clustering = nx.average_clustering(G)
        if len(distrib_hist) > 10:
            answer.append('PA')
        elif clustering < 0.1:
            answer.append('SW_H')
        else:
            answer.append('SW_L')
        
    return answer

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def salary_predictions():
    
    df = pd.DataFrame(index=G.nodes())
    df['Management'] = pd.Series(nx.get_node_attributes(G, "ManagementSalary"))
    df['Department'] = pd.Series(nx.get_node_attributes(G, "Department"))
    df['Close_cent'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['Betweeen_cent'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    df['Degree'] = pd.Series(G.degree())
    df['Degree_cent'] = pd.Series(nx.degree_centrality(G))
    df['PageRank'] = pd.Series(nx.pagerank(G, alpha=0.85))
    
    y = df['Management']
    y_train = y[y.notnull()]
    
    X = df.drop(['Management'], axis=1)
    X_train = X[df.Management.notnull()]
    X_test = X[df.Management.isnull()]
    
#     scores = []
#     d, e = [], []
#     for depth in range(3, 6):
#         for estimator in [75, 100, 125, 150]:
#             rfc = RandomForestClassifier(n_estimators=estimator, max_depth=depth)
#             scores.append(np.mean(cross_val_score(rfc, X_train, y_train, cv=5, scoring='roc_auc')))
#             print(scores[-1])
#             d.append(depth)
#             e.append(estimator)
#     index = scores.index(max(scores))
#     print("for depth {} and {} estimators score is {}".format(d[index], e[index], scores[index]))

    rfc = RandomForestClassifier(n_estimators=100, max_depth=3)
    rfc.fit(X_train, y_train)
    
    proba = rfc.predict_proba(X_test)
    answer = pd.Series([x[1] for x in proba], index=X_test.index)
    
    return answer

salary_predictions()

1       0.198590
2       0.887276
5       0.996539
8       0.207505
14      0.207489
18      0.226964
27      0.232438
30      0.202263
31      0.225628
34      0.061896
37      0.110858
40      0.211621
45      0.207730
54      0.234747
55      0.244221
60      0.162796
62      0.996539
65      0.873754
77      0.043577
79      0.225485
97      0.028358
101     0.059237
103     0.237727
108     0.234853
113     0.254337
122     0.044177
141     0.380750
142     0.996539
144     0.046864
145     0.206570
          ...   
913     0.028273
914     0.028931
915     0.027059
918     0.044262
923     0.024869
926     0.028923
931     0.027586
934     0.030903
939     0.025243
944     0.027059
945     0.025243
947     0.024858
950     0.243426
951     0.024586
953     0.025707
959     0.026630
962     0.025243
963     0.209505
968     0.025516
969     0.027543
974     0.024981
984     0.032930
987     0.024752
989     0.025516
991     0.031175
992     0.025087
994     0.027059
996     0.0270

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [20]:
def new_connections_predictions():
    
    df = future_connections
    df['pref_attach'] = [item[2] for item in nx.preferential_attachment(G, df.index)]
    df['jaccard'] = [coef[2] for coef in nx.jaccard_coefficient(G, df.index)]
    df['res_alloc_ind'] = [index[2] for index in nx.resource_allocation_index(G, df.index)]
    df['adamic_adar'] = [index[2] for index in nx.adamic_adar_index(G, df.index)]
    df['com_neigh'] = df.index.map(lambda node: len(list(nx.common_neighbors(G, node[0], node[1]))))
    
    
    y = df['Future Connection']
    y_train = y[y.notnull()]
    
    X = df.drop(['Future Connection'], axis=1)
    X_train = X[y.notnull()]
    X_test = X[y.isnull()]
    
    
    rfc = RandomForestClassifier(n_estimators=100, max_depth=3)
    rfc.fit(X_train, y_train)
    
    proba = rfc.predict_proba(X_test)
    answer = pd.Series([x[1] for x in proba], index=X_test.index)
    
    return answer

new_connections_predictions()

(107, 348)    0.020422
(542, 751)    0.017614
(20, 426)     0.570570
(50, 989)     0.017614
(942, 986)    0.017614
(324, 857)    0.017614
(13, 710)     0.196027
(19, 271)     0.154519
(319, 878)    0.017614
(659, 707)    0.017614
(49, 843)     0.017614
(208, 893)    0.017614
(377, 469)    0.020671
(405, 999)    0.017614
(129, 740)    0.017614
(292, 618)    0.096434
(239, 689)    0.017614
(359, 373)    0.019873
(53, 523)     0.019733
(276, 984)    0.017614
(202, 997)    0.017614
(604, 619)    0.094778
(270, 911)    0.017614
(261, 481)    0.056363
(200, 450)    0.871966
(213, 634)    0.017614
(644, 735)    0.099833
(346, 553)    0.017614
(521, 738)    0.017614
(422, 953)    0.017614
                ...   
(672, 848)    0.017614
(28, 127)     0.931654
(202, 661)    0.017614
(54, 195)     0.950272
(295, 864)    0.017614
(814, 936)    0.017614
(839, 874)    0.017614
(139, 843)    0.017614
(461, 544)    0.017614
(68, 487)     0.017614
(622, 932)    0.017614
(504, 936)    0.017838
(479, 528) 